# PROJET IMAGE

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.svm import SVR
import torch
import torch.nn as nn
import torch.optim as optim
from skimage.feature import hog
from skimage import color
loo=LeaveOneOut()

In [8]:
# drive.mount('/content/drive')
os.chdir('../data/raw')

In [9]:
def generer_donnees():
    groupes = {
        'Groupe2': ['jpeg', 'jpeg', 'jpeg', 'png', 'jpeg', 'jpeg', 'jpeg', 'jpg', 'jpg', 'jpg', 'jpg', 'png', 'png'],
        'Groupe3': ['jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpeg', 'jpg', 'jpg', 'jpeg', 'jpg', 'png', 'png', 'png', 'png', 'png', 'jpeg', 'png', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg'],
        'Groupe6': ['jpg'] * 10,
        'Groupe1': ['jpg'] * 10,
        'Groupe4': ['png', 'jpg', 'png', 'jpg', 'jpg', 'jpg', 'png', 'png', 'png'],
        'Groupe7': ['jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpg', 'jpg', 'jpg'],
        'Groupe5': ['jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpg', 'jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpeg', 'jpeg']
    }

    valeurs = {
        'Groupe2': [4, 7, 9, 6, 8, 10, 10, 9, 9, 10, 9, 7, 16],
        'Groupe3': [11, 12, 7, 10, 5, 4, 22, 17, 21, 34, 6, 5, 4, 4, 5, 4, 3, 3, 4, 2, 14, 7, 12, 10, 7, 8, 7, 8, 12, 7],
        'Groupe6': [15, 7, 11, 9, 7, 9, 7, 10, 10, 8],
        'Groupe1': [11, 8, 13, 12, 12, 6, 6, 17, 9, 15],
        'Groupe4': [8, 10, 22, 37, 5, 11, 11, 10, 12],
        'Groupe7': [15, 11, 15, 2, 10, 10, 6, 22],
        'Groupe5': [9, 4, 38, 6, 7, 5, 11, 14, 8, 14, 13, 15, 3, 4, 8, 10, 13, 8]
    }

    result = {}

    for groupe, extensions in groupes.items():
        result[groupe] = {f"{groupe}_image{str(i+1).zfill(2)}.{ext}": val for i, (ext, val) in enumerate(zip(extensions, valeurs[groupe]))}

    return result

def predire_nombre_escaliers(image_path, scaler, modele_rf,Ajoute_Features,type_filtre):
    try:
        caracteristiques = Vecteur_Caracteristique(image_path,ajouter_caracteristiques=Ajoute_Features,type_filtre=type_filtre)
        caracteristiques_echellees = scaler.transform([caracteristiques])
        prediction = modele_rf.predict(caracteristiques_echellees)
        nombre_escaliers = round(prediction[0])
        print(f"Nombre d'escaliers détectés : {nombre_escaliers}")
    except Exception as erreur:
        print(f"Erreur de prédiction : {erreur}")


def predire_nombre_escaliers2(image_path, scaler, modele_rf,Ajoute_Features,type_filtre):
    try:
        caracteristiques = Vecteur_Caracteristique(image_path,ajouter_caracteristiques=Ajoute_Features,type_filtre=type_filtre)
        caracteristiques_echellees = scaler.transform([caracteristiques])
        caracteristiques_echellees = torch.tensor(caracteristiques_echellees, dtype=torch.float32).to(device)

        modele_rf.eval()
        with torch.no_grad():
            prediction = modele_rf(caracteristiques_echellees).detach().cpu().numpy().flatten()[0]
        nombre_escaliers = round(prediction[0])
        print(f"Nombre d'escaliers détectés : {nombre_escaliers}")
    except Exception as erreur:
        print(f"Erreur de prédiction : {erreur}")


def Vecteur_Caracteristique(image_path, bins=256, hauteur_image=1024, type_filtre=1, ajouter_caracteristiques=0,Add_Hog=False,hog_orientations=8, color_bins=32):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Erreur lors du chargement de l'image : {image_path}")
        return None
    image_redim = cv2.resize(image, (1024, hauteur_image))

    gris = cv2.cvtColor(image_redim, cv2.COLOR_BGR2GRAY)

    if type_filtre == 1:
        edges = cv2.Canny(gris, 50, 150, apertureSize=3)
    elif type_filtre == 2:
        sobel_x = cv2.Sobel(gris, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gris, cv2.CV_64F, 0, 1, ksize=3)
        magnitude_sobel = cv2.magnitude(sobel_x, sobel_y)
        edges = cv2.convertScaleAbs(magnitude_sobel)
    elif type_filtre == 3:
        gris_flou = cv2.GaussianBlur(gris, (5, 5), 0)
        edges = cv2.Canny(gris_flou, 50, 150, apertureSize=3)
    elif type_filtre == 4:
        gris_bilateral = cv2.bilateralFilter(gris, 9, 75, 75)
        edges = cv2.Canny(gris_bilateral, 50, 150, apertureSize=3)
    elif type_filtre == 5:
        gris_bilateral = cv2.bilateralFilter(gris, 9, 75, 75)
        sobel_x = cv2.Sobel(gris_bilateral, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gris_bilateral, cv2.CV_64F, 0, 1, ksize=3)
        magnitude_sobel = cv2.magnitude(sobel_x, sobel_y)
        edges = cv2.convertScaleAbs(magnitude_sobel)

    lignes_detectees = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=120, maxLineGap=15)

    Vecteur = np.zeros(bins)

    longueurs_lignes = []
    if lignes_detectees is not None:
      angles_lignes_horizontales = []
      angles_lignes_verticales = []
      compteur_horizontal = 0
      compteur_vertical = 0

      for ligne in lignes_detectees:
          x1, y1, x2, y2 = ligne[0]
          angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))

          if abs(angle) < 10:
              compteur_horizontal += 1
              angles_lignes_horizontales.append(angle)

              y_centre = (y1 + y2) // 2
              index_bin = y_centre // 4
              Vecteur[index_bin] += 1

              if ajouter_caracteristiques:
                  longueur = np.hypot(x2 - x1, y2 - y1)
                  longueurs_lignes.append(longueur)

          elif abs(angle - 90) < 10:
              compteur_vertical += 1
              angles_lignes_verticales.append(angle)


    variance_angle_horizontal = np.var(angles_lignes_horizontales) if angles_lignes_horizontales else 0
    variance_angle_vertical = np.var(angles_lignes_verticales) if angles_lignes_verticales else 0

    if ajouter_caracteristiques:
        moyenne_longueur = np.mean(longueurs_lignes) if longueurs_lignes else 0
        ecart_type_longueur = np.std(longueurs_lignes) if longueurs_lignes else 0

        features_supplementaires = np.array([
            compteur_horizontal,
            compteur_vertical,
            variance_angle_horizontal,
            variance_angle_vertical,
            moyenne_longueur,
            ecart_type_longueur
        ])

        return np.concatenate([Vecteur, features_supplementaires])
    # vecteur2=[]
    # vecteur2.extend(Vecteur)
    # if Add_Hog:
    #   print("HOG")
    #   hog_features, _ = hog(gris, orientations=hog_orientations, pixels_per_cell=(16, 16),
    #                       cells_per_block=(1, 1), visualize=True, feature_vector=True)
    #   vecteur2.extend(hog_features)

    #
    #   hsv_image = cv2.cvtColor(image_redim, cv2.COLOR_BGR2HSV)
    #   hist_h = cv2.calcHist([hsv_image], [0], None, [color_bins], [0, 180]).flatten()
    #   hist_s = cv2.calcHist([hsv_image], [1], None, [color_bins], [0, 256]).flatten()
    #   hist_v = cv2.calcHist([hsv_image], [2], None, [color_bins], [0, 256]).flatten()

    #   hist_h = hist_h / hist_h.sum()
    #   hist_s = hist_s / hist_s.sum()
    #   hist_v = hist_v / hist_v.sum()

    #   vecteur2.extend(hist_h)
    #   vecteur2.extend(hist_s)
    #   vecteur2.extend(hist_v)

    #   return np.array(vecteur2)

    return Vecteur




class Net(nn.Module):
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout_rate=0.3):
        super(Net, self).__init__()
        layers = []
        previous_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(previous_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout_rate))
            previous_dim = hidden_dim

        layers.append(nn.Linear(previous_dim, 1))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)



In [12]:
import cv2
import numpy as np

def merge_close_lines_auto2(y_lines, base_factor=0.5, min_thresh=5):
    if not y_lines:
        return []

    # Sort the y-coordinates in ascending order
    y_lines = sorted(y_lines)

    if len(y_lines) < 2:
        return y_lines

    # Compute the differences between consecutive lines
    diffs = np.diff(y_lines)
    mean_gap = np.mean(diffs)
    std_gap = np.std(diffs)

    threshold = base_factor * mean_gap + std_gap
    threshold = max(threshold, min_thresh)  # Ensure the threshold is at least min_thresh

    merged_lines = []
    current_group = [y_lines[0]]

    for y in y_lines[1:]:
        if y - current_group[-1] <= threshold:
            current_group.append(y)
        else:
            merged_lines.append(int(round(np.mean(current_group))))
            current_group = [y]

    merged_lines.append(int(round(np.mean(current_group))))
    return merged_lines

def count_stairs(image_path):
    # 1. Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image from {image_path}")
        return 0

    # 2. Convert to grayscale and detect edges
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, threshold1=50, threshold2=150)

    # Optional morphological closing to refine edges
    kernel = np.ones((3, 3), np.uint8)
    closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=1)

    # 3. Detect lines using HoughLinesP
    lines = cv2.HoughLinesP(
        closed_edges,
        rho=1,
        theta=np.pi / 180,
        threshold=120,
        minLineLength=125,
        maxLineGap=5
    )

    if lines is None:
        print("No lines were detected.")
        return 0

    # 4. Filter out near-horizontal lines and collect their y-coordinates
    line_image = img.copy()
    horizontal_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.arctan2((y2 - y1), (x2 - x1)) * 180.0 / np.pi
        if abs(angle) < 10:  # nearly horizontal
            mean_y = int(round((y1 + y2) / 2))
            horizontal_lines.append(mean_y)
            # (Optional) draw the raw lines in green
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 1)

    # 5. Sort and remove duplicates
    horizontal_lines = sorted(set(horizontal_lines))

    # 6. Merge lines that are very close
    merged_lines = merge_close_lines_auto2(horizontal_lines, base_factor=0.5, min_thresh=5)

    # 7. Draw the merged lines in red
    for y in merged_lines:
        cv2.line(line_image, (0, y), (img.shape[1], y), (0, 0, 255), 2)

    num_stairs = len(merged_lines)

    print(f"Detected {num_stairs} stair lines after merging close lines.")

    cv2.imshow("Detected Stairs (Resized)", line_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite("stair_detection_result.jpg", line_image)

    return num_stairs

if __name__ == "__main__":
    image_file = "Groupe1_Image1.jpg"
    stairs_count = count_stairs(image_file)
    print("Number of stairs detected:", stairs_count)

Detected 9 stair lines after merging close lines.
Number of stairs detected: 9


# Get Data

In [14]:
data=generer_donnees()
path = './'
Groupe1 = data['Groupe1']
Groupe2 = data['Groupe2']
Groupe3 = data['Groupe3']
Groupe4 = data['Groupe4']
Groupe5 = data['Groupe5']
Groupe6 = data['Groupe6']
Groupe7 = data['Groupe7']

groups = {'Groupe 1':Groupe1, 'Groupe 2':Groupe2, 'Groupe 3':Groupe3, 'Groupe 4':Groupe4, 'Groupe 5':Groupe5, 'Groupe 6':Groupe6, 'Groupe 7':Groupe7}

In [ ]:
data

# Gradient Boost

In [13]:

params_gb = [
    {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 4},
    {'n_estimators': 1000, 'learning_rate': 0.1, 'max_depth': 6},
    {'n_estimators': 2000, 'learning_rate': 0.05, 'max_depth': 3},
]

with open('Historique_GradientBoosting.txt', 'w') as file:

    for m in range(1, 6):
        for z in range(2):
            X = []
            y = []
            for group in groups.keys():
                group_path = os.path.join(path, group)
                for image_name in os.listdir(group_path):
                    if image_name not in groups[group].keys():
                        print("on est la " + image_name)
                    image_path = os.path.join(group_path, image_name)
                    features = Vecteur_Caracteristique(image_path, filtre=m, addfeatures=z)
                    if features is not None:
                        X.append(features)
                        y.append(groups[group][image_name])

            X = np.array(X)
            y = np.array(y)

            for n in range(3):
                if n == 0:
                    scaler = StandardScaler()
                    X_scaled = scaler.fit_transform(X)
                elif n == 1:
                    scaler = PowerTransformer()
                    X_scaled = scaler.fit_transform(X)
                elif n == 2:
                    scaler = RobustScaler()
                    X_scaled = scaler.fit_transform(X)

                for param in params_gb:
                    loo = LeaveOneOut()
                    errors_gb = []
                    gb = GradientBoostingRegressor(random_state=42, **param)

                    for train_index, test_index in loo.split(X_scaled):
                        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
                        y_train, y_test = y[train_index], y[test_index]

                        gb.fit(X_train, y_train)
                        print("pass")
                        y_pred_gb = gb.predict(X_test)
                        errors_gb.append(abs(round(y_test[0]) - y_pred_gb[0]))

                    mean_error_gb = np.mean(errors_gb)

                    file.write(f"Filtre {m}, Paramètre {param}, AddFeatures {z}, Erreur absolue moyenne (Gradient Boosting) pour le scaler {['StandardScaler', 'PowerTransformer', 'RobustScaler'][n]} : {mean_error_gb}\n")

print("Les résultats ont été enregistrés dans 'Historique_GradientBoosting.txt'.")


NameError: name 'groups' is not defined

# XGBOOST

In [ ]:
params_xgb = [
    {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 4},
    {'n_estimators': 1000, 'learning_rate': 0.1, 'max_depth': 6},
    {'n_estimators': 2000, 'learning_rate': 0.05, 'max_depth': 3},
]

with open('Historique_XGBoost.txt', 'w') as file:
    for m in range(1, 6):
        for z in range(2):
            X = []
            y = []
            for group in groups.keys():
                group_path = os.path.join(path, group)
                for image_name in os.listdir(group_path):
                    if image_name not in groups[group].keys():
                        print("on est la " + image_name)
                    image_path = os.path.join(group_path, image_name)
                    features = Vecteur_Caracteristique(image_path, filtre=m, addfeatures=z)
                    if features is not None:
                        X.append(features)
                        y.append(groups[group][image_name])

            X = np.array(X)
            y = np.array(y)
            for n in range(3):
                if n == 0:
                    scaler = StandardScaler()
                    X_scaled = scaler.fit_transform(X)
                elif n == 1:
                    scaler = PowerTransformer()
                    X_scaled = scaler.fit_transform(X)
                elif n == 2:
                    scaler = RobustScaler()
                    X_scaled = scaler.fit_transform(X)

                for param in params_xgb:
                    loo = LeaveOneOut()
                    errors_xgb = []

                    xgb_model = xgb.XGBRegressor(random_state=42, **param)
                    for train_index, test_index in loo.split(X_scaled):
                        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
                        y_train, y_test = y[train_index], y[test_index]
                        xgb_model.fit(X_train, y_train,xgb_model=xgb_model)
                        print("pass")
                        y_pred_xgb = xgb_model.predict(X_test)
                        errors_xgb.append(abs(round(y_test[0]) - y_pred_xgb[0]))

                    mean_error_xgb = np.mean(errors_xgb)

                    file.write(f"Filtre {m}, Paramètre {param}, AddFeatures {z}, Erreur absolue moyenne (XGBoost) pour le scaler {['StandardScaler', 'PowerTransformer', 'RobustScaler'][n]} : {mean_error_xgb}\n")

print("Les résultats ont été enregistrés dans 'Historique_XGBoost.txt'.")

# SVR

In [ ]:
params_svr = [
    {'C': 0.1, 'epsilon': 0.01, 'kernel': 'rbf'},
    {'C': 0.5, 'epsilon': 0.05, 'kernel': 'rbf'},
    {'C': 1.0, 'epsilon': 0.1, 'kernel': 'rbf'},
    {'C': 10.0, 'epsilon': 0.05, 'kernel': 'rbf'},
    {'C': 10.0, 'epsilon': 0.1, 'kernel': 'rbf'},
    {'C': 100.0, 'epsilon': 0.2, 'kernel': 'rbf'},
    {'C': 1.0, 'epsilon': 0.1, 'kernel': 'linear'},
    {'C': 10.0, 'epsilon': 0.05, 'kernel': 'linear'},
    {'C': 100.0, 'epsilon': 0.1, 'kernel': 'linear'},
    {'C': 1.0, 'epsilon': 0.1, 'kernel': 'poly'},
    {'C': 10.0, 'epsilon': 0.05, 'kernel': 'poly'},
    {'C': 100.0, 'epsilon': 0.1, 'kernel': 'poly'},
    {'C': 1000.0, 'epsilon': 0.2, 'kernel': 'poly'},
]

with open('Historique_SVR.txt', 'w') as file:

    for m in range(1, 6):
        for z in range(2):
            X = []
            y = []
            for group in groups.keys():
                group_path = os.path.join(path, group)
                for image_name in os.listdir(group_path):
                    if image_name not in groups[group].keys():
                        print("on est la " + image_name)
                    image_path = os.path.join(group_path, image_name)
                    features = Vecteur_Caracteristique(image_path, filtre=m, addfeatures=z)
                    if features is not None:
                        X.append(features)
                        y.append(groups[group][image_name])

            X = np.array(X)
            y = np.array(y)

            for n in range(3):
                if n == 0:
                    scaler = StandardScaler()
                    X_scaled = scaler.fit_transform(X)
                elif n == 1:
                    scaler = PowerTransformer()
                    X_scaled = scaler.fit_transform(X)
                elif n == 2:
                    scaler = RobustScaler()
                    X_scaled = scaler.fit_transform(X)

                for param in params_svr:
                    loo = LeaveOneOut()
                    errors_svr = []
                    svr_model = SVR(**param)

                    for train_index, test_index in loo.split(X_scaled):
                        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
                        y_train, y_test = y[train_index], y[test_index]

                        svr_model.fit(X_train, y_train)
                        print("pass")
                        y_pred_svr = svr_model.predict(X_test)
                        errors_svr.append(abs(round(y_test[0]) - y_pred_svr[0]))
                    mean_error_svr = np.mean(errors_svr)


                    file.write(f"Filtre {m}, Paramètre {param}, AddFeatures {z}, Erreur absolue moyenne (SVR) pour le scaler {['StandardScaler', 'PowerTransformer', 'RobustScaler'][n]} : {mean_error_svr}\n")

print("Les résultats ont été enregistrés dans 'Historique_SVR.txt'.")


# RESEAUX NEURONNE

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du device : {device}")

# Paramètres à tester pour les réseaux de neurones
hidden_dims=[256, 128, 64]
num_layers = [2, 3, 4,8]
dropout_rates = [0.2, 0.3, 0.4]

# Initialisation du fichier pour enregistrer les résultats
with open('Historique_NeuralNetwork.txt', 'w') as file:
    for m in range(1, 6):
        for z in range(2):
            X = []
            y = []
            for group in groups.keys():
                group_path = os.path.join(path, group)
                for image_name in os.listdir(group_path):
                    if image_name not in groups[group].keys():
                        print("on est la " + image_name)
                    image_path = os.path.join(group_path, image_name)
                    features = Vecteur_Caracteristique(image_path, filtre=m, addfeatures=z)
                    if features is not None:
                        X.append(features)
                        y.append(groups[group][image_name])

            X = np.array(X)
            y = np.array(y)

            for n in range(3):
                if n == 0:
                    scaler = StandardScaler()
                    X_scaled = scaler.fit_transform(X)
                elif n == 1:
                    scaler = PowerTransformer()
                    X_scaled = scaler.fit_transform(X)
                elif n == 2:
                    scaler = RobustScaler()
                    X_scaled = scaler.fit_transform(X)

                for hidden_dim in hidden_dims:
                    for layers in num_layers:
                        for dropout_rate in dropout_rates:
                            loo = LeaveOneOut()
                            errors_nn = []

                            input_dim = X_scaled.shape[1]
                            epochs = 50
                            learning_rate = 0.01
                            for train_index, test_index in loo.split(X_scaled):
                                X_train, X_test = X_scaled[train_index], X_scaled[test_index]
                                y_train, y_test = y[train_index], y[test_index]

                                X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
                                y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
                                X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

                                model = Net(input_dim, hidden_dim, layers, dropout_rate).to(device)
                                optimizer = optim.Adam(model.parameters(), lr=learning_rate)
                                criterion = nn.MSELoss()

                                model.train()
                                for epoch in range(epochs):
                                    optimizer.zero_grad()
                                    outputs = model(X_train_tensor)
                                    loss = criterion(outputs, y_train_tensor)
                                    loss.backward()
                                    optimizer.step()
                                print("train")
                                model.eval()
                                with torch.no_grad():
                                    y_pred = model(X_test_tensor).cpu().numpy().flatten()[0]

                                errors_nn.append(abs(round(y_test[0]) - y_pred))

                            mean_error_nn = np.mean(errors_nn)

                            file.write(f"Filtre: {m}, AddFeatures: {z}, Normalisation: {['StandardScaler', 'PowerTransformer', 'RobustScaler'][n]}, "
                                       f"Hidden Dim: {hidden_dim}, Layers: {layers}, Dropout Rate: {dropout_rate}, "
                                       f"Erreur absolue moyenne (Réseau de Neurones PyTorch) : {mean_error_nn}\n")

print("Les résultats ont été enregistrés dans 'Historique_NeuralNetwork.txt'.")


#  Decision Tree

In [ ]:
params = [
    {'n_estimators': 1000, 'max_depth': None, 'min_samples_split': 2},
    {'n_estimators': 500, 'max_depth': 10, 'min_samples_split': 5},
    {'n_estimators': 2000, 'max_depth': 20, 'min_samples_split': 2},
]

with open('Historique_RandomForest.txt', 'w') as file:
  for m in range(1,6):
    for z in range(2):
      X = []
      y = []
      for group in groups.keys():
          group_path = os.path.join(path, group)
          for image_name in os.listdir(group_path):
            if image_name not in groups[group].keys() :
              print("on est la "+image_name)
            image_path = os.path.join(group_path, image_name)
            features = Vecteur_Caracteristique(image_path,filtre=m,addfeatures=z)
            if features is not None:
                X.append(features)
                y.append(groups[group][image_name])

      X = np.array(X)
      y = np.array(y)

      for n in range(3):
        if n==0:
          scaler = StandardScaler()
          X_scaled = scaler.fit_transform(X)
        elif n==1:
          scaler = PowerTransformer()
          X_scaled = scaler.fit_transform(X)
        elif n==2:
          scaler = RobustScaler()
          X_scaled = scaler.fit_transform(X)

        for param in params:
          loo = LeaveOneOut()
          errors_rf = []
          rf = RandomForestRegressor(random_state=42,warm_start=True, **param)

          # Boucle LOO
          for train_index, test_index in loo.split(X_scaled):
              X_train, X_test = X_scaled[train_index], X_scaled[test_index]
              y_train, y_test = y[train_index], y[test_index]
              rf.n_estimators += 5
              if len(X_train) > 1:
                  rf.fit(X_train, y_train)

              y_pred_rf = rf.predict(X_test)
              errors_rf.append(abs(round(y_test[0]) - y_pred_rf[0]))

          # Résultats
          mean_error_rf = np.mean(errors_rf)
          print("done")
          file.write(f"Filtre {m} , et parametre {param},addfeatures{z}, Erreur absolue moyenne (Random Forest) pour le scaler {['StandardScaler', 'PowerTransformer', 'RobustScaler'][n]} : {mean_error_rf}\n")

# Train Model

In [ ]:
data

In [ ]:
def Prepare_data(scaler,Add_features,type_filtre):
  X = []
  y = []
  for group in groups.keys():
      group_path = os.path.join(path, group)
      for image_name in os.listdir(group_path):
        image_path = os.path.join(group_path, image_name)
        features = Vecteur_Caracteristique(image_path,ajouter_caracteristiques=Add_features,type_filtre=type_filtre)
        if features is not None:
            X.append(features)
            y.append(groups[group][image_name])

  X = np.array(X)
  y = np.array(y)
  X_scaled = scaler.fit_transform(X)

  return X_scaled, y


def Train_model(modele, X, y,RF=False):
    loo = LeaveOneOut()
    erreurs = []

    for train_idx, test_idx in loo.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        if RF:
          modele.n_estimators += 5
        if len(X_train) > 1:
          modele.fit(X_train, y_train)
          y_pred = modele.predict(X_test)
          erreurs.append(abs(round(y_test[0]) - y_pred[0]))

    erreur_moyenne = np.mean(erreurs)
    print(f"Erreur absolue moyenne ({type(modele).__name__}) : {erreur_moyenne}")
    return modele


# xgb_model.fit(X_train, y_train,xgb_model=xgb_model)
Xgboost_params={'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 4}
My_model = xgb.XGBRegressor(random_state=42, **Xgboost_params)
Ajoute_Features=1
type_filtre=1
scaler = StandardScaler() # Choisir la methode de normalisation
X, y = Prepare_data(scaler,Ajoute_Features,type_filtre) # Recuperer les donner
# My_model = RandomForestRegressor(n_estimators=1000, random_state=42,warm_start=True) # Creer un Model de regression
My_model=Train_model(My_model, X, y,RF=False) # entrainement du model



# Evaluation

In [ ]:
Test_image = './Groupe 3/Groupe3_image16.png'
Test_image = './escalier4.jpg'
predire_nombre_escaliers(Test_image,scaler,My_model,Ajoute_Features,type_filtre)

Nombre d'escaliers détectés : 9


9

In [ ]:
Groupe3

# Train Neurale network

In [ ]:
type_filtre=1
Ajoute_Features=1
scaler = StandardScaler()
X, y = Prepare_data(scaler,Ajoute_Features,type_filtre) # Recuperer les donner

hidden_dims_final = [256, 128, 64]
num_layers_final = 3
dropout_rate_final = 0.3
normalisation_finale = StandardScaler()
filtre_final = 3
addfeatures_final = 1

input_dim_final = X.shape[1]
epochs_final = 100
learning_rate_final = 0.005
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
errors_nn_final = []


for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

    My_model = Net(input_dim_final, hidden_dims_final, dropout_rate_final).to(device)
    optimizer = optim.Adam(My_model.parameters(), lr=learning_rate_final)
    criterion = nn.MSELoss()

    My_model.train()
    for epoch in range(epochs_final):
        optimizer.zero_grad()
        outputs = My_model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

    My_model.eval()
    with torch.no_grad():
        y_pred = My_model(X_test_tensor).cpu().numpy().flatten()[0]

    errors_nn_final.append(abs(round(y_test[0]) - y_pred))

mean_error_nn_final = np.mean(errors_nn_final)

# Evaluation

In [ ]:
Test_image = './Groupe 3/Groupe3_image14.jpeg'
Test_image = './escalier.jpg'
predire_nombre_escaliers2(Test_image,scaler,My_model,Ajoute_Features,type_filtre)
mean_error_nn_final

np.float32(5.0657544)

In [ ]:
Groupe3

# Evaluation

# Perspective
**Applique la segmentation avec yolo pour recuperer uniquemtn les escalier **

In [ ]:
import os
import re

#Regler les Donner
groupes = {
    "Groupe 1": "Groupe1_image",
    "Groupe 2": "Groupe2_image",
    "Groupe 3": "Groupe3_image",
    "Groupe 4": "Groupe4_image",
    "Groupe 5": "Groupe5_image",
    "Groupe 6": "Groupe6_image",
    "Groupe 7": "Groupe7_image"
}

def extraire_numero(fichier):
    numeros = re.findall(r'\d+', fichier)
    return int(numeros[-1]) if numeros else 0


def renommer_images(chemin_groupe, nouveau_prefixe):
    fichiers = os.listdir(chemin_groupe)
    fichiers_tries = sorted(fichiers, key=extraire_numero)

    compteur = 1
    for fichier in fichiers_tries:
        ext = fichier.split('.')[-1]
        nouveau_nom = f"{nouveau_prefixe}{str(compteur).zfill(2)}.{ext}"

        ancien_chemin = os.path.join(chemin_groupe, fichier)
        nouveau_chemin = os.path.join(chemin_groupe, nouveau_nom)

        os.rename(ancien_chemin, nouveau_chemin)
        print(f"{fichier} ➜ {nouveau_nom}")

        compteur += 1

# Renommer chaque groupe correctement
for groupe, prefixe in groupes.items():
    chemin = os.path.join("./", groupe)
    renommer_images(chemin, prefixe)